In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/flipkart-dataset/Dataset-SA.csv


In [2]:
df = pd.read_csv("/kaggle/input/flipkart-dataset/Dataset-SA.csv")

In [3]:
df.head()

,product_name,product_price,Rate,Review,Summary,Sentiment
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import ast
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
df = df[df['Summary'].notnull()]

In [6]:
categories = []
for x in df['Sentiment']:
    y = x.split()
    for item in y:
        categories.append(item)

In [7]:
categories = list(dict.fromkeys(categories))

In [8]:
categories

['positive', 'negative', 'neutral']

In [9]:
df2 = pd.DataFrame(columns=categories)

In [10]:
# one-hot-encoding the categories
for x, idx in zip(df['Sentiment'], range(0, df.shape[0])):
    y = x.split()
    row = []
    for item in df2.columns:
        if item in y:
            row.append(1)
        else:
            row.append(0)
    df2.loc[idx] = row

In [11]:
df2

,positive,negative,neutral
0,1,0,0
1,1,0,0
2,1,0,0
3,0,1,0
4,0,0,1
...,...,...,...
205036,1,0,0
205037,1,0,0
205038,1,0,0
205039,1,0,0


In [12]:
df3 = pd.concat([df, df2], axis=1).reindex(df2.index)

In [13]:
df3


,product_name,product_price,Rate,Review,Summary,Sentiment,positive,negative,neutral
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,positive,1.0,0.0,0.0
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,positive,1.0,0.0,0.0
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,positive,1.0,0.0,0.0
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,negative,0.0,1.0,0.0
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,neutral,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
205036,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,great product,good product,positive,1.0,0.0,0.0
205037,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,small but good,positive,1.0,0.0,0.0
205038,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,fabulous!,nice i gifted it,positive,1.0,0.0,0.0
205039,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,superb i really like it,positive,1.0,0.0,0.0


In [14]:
final_df = df3.drop(['Sentiment'], axis=1)

In [15]:
final_df

,product_name,product_price,Rate,Review,Summary,positive,negative,neutral
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excellent air flow and for this p...,1.0,0.0,0.0
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget 2 fit cooler nice cooling,1.0,0.0,0.0
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,the quality is good but the power of air is de...,1.0,0.0,0.0
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,very bad product its a only a fan,0.0,1.0,0.0
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
205036,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,great product,good product,1.0,0.0,0.0
205037,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,small but good,1.0,0.0,0.0
205038,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,fabulous!,nice i gifted it,1.0,0.0,0.0
205039,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,superb i really like it,1.0,0.0,0.0


In [16]:
# pre-processing the data
def decontract(sentence):
    # specific
    sentence = re.sub(r"won't", "will not", str(sentence))
    sentence = re.sub(r"can\'t", "can not", sentence)

    # general
    sentence = re.sub(r"n\'t", " not", sentence)
    sentence = re.sub(r"\'re", " are", sentence)
    sentence = re.sub(r"\'s", " is", sentence)
    sentence = re.sub(r"\'d", " would", sentence)
    sentence = re.sub(r"\'ll", " will", sentence)
    sentence = re.sub(r"\'t", " not", sentence)
    sentence = re.sub(r"\'ve", " have", sentence)
    sentence = re.sub(r"\'m", " am", sentence)
    return sentence

def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', '', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub("", sentence)

In [17]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

re_stop_words = re.compile(r"\b(" + "|".join(stopwords) + ")\\W", re.I)

In [18]:
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

In [19]:
final_df['Summary'] = final_df['Summary'] + " The review is " + final_df['Review']

In [20]:
final_df['Summary'] = final_df['Summary'].str.lower()
final_df['Summary'] = final_df['Summary'].apply(decontract)
final_df['Summary'] = final_df['Summary'].apply(cleanPunc)
final_df['Summary'] = final_df['Summary'].apply(keepAlpha)
final_df['Summary'] = final_df['Summary'].apply(removeStopWords)
final_df['Summary'] = final_df['Summary'].apply(stemming)

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=50000, lower=True)
tokenizer.fit_on_texts(final_df['Summary'])
sequences = tokenizer.texts_to_sequences(final_df['Summary'])
x = pad_sequences(sequences, maxlen=200)

In [22]:
final_df

,product_name,product_price,Rate,Review,Summary,positive,negative,neutral
0,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,super!,great cooler excel air flow price amaz unbelie...,1.0,0.0,0.0
1,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,5,awesome,best budget fit cooler nice cool review awesom,1.0,0.0,0.0
2,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,qualiti good power air decent review fair,1.0,0.0,0.0
3,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,1,useless product,bad product fan review useless product,0.0,1.0,0.0
4,Candes 12 L Room/Personal Air Cooler??????(Whi...,3999,3,fair,ok ok product review fair,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
205036,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,great product,good product review great product,1.0,0.0,0.0
205037,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,small good review terrif,1.0,0.0,0.0
205038,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,fabulous!,nice gift review fabul,1.0,0.0,0.0
205039,cello Pack of 18 Opalware Cello Dazzle Lush Fi...,1299,5,terrific,superb realli like review terrif,1.0,0.0,0.0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(x, 
                                                    final_df[final_df.columns[5:]], 
                                                    test_size=0.2)

In [24]:
y_train

,positive,negative,neutral
143712,1.0,0.0,0.0
177147,0.0,1.0,0.0
146528,1.0,0.0,0.0
55269,1.0,0.0,0.0
98152,1.0,0.0,0.0
...,...,...,...
179774,0.0,1.0,0.0
143968,1.0,0.0,0.0
110291,1.0,0.0,0.0
49420,1.0,0.0,0.0


In [25]:
num_classes = y_train.shape[1]
max_words = len(tokenizer.word_index) + 1
maxlen = 200

In [26]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [27]:
num_classes

3

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LeakyReLU, Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import tensorflow as tf

# Increase model complexity by adding more layers and neurons
filter_length = 300

model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))  # Increased embedding dimension
model.add(Conv1D(128, 5, padding='valid', activation=LeakyReLU(alpha=0.01), strides=1))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.2))
model.add(Dense(256, activation=LeakyReLU(alpha=0.01)))
model.add(Dropout(0.1))
model.add(Dense(100, activation=LeakyReLU(alpha=0.01)))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

callbacks = [
    ReduceLROnPlateau(patience=2),  # Reduce learning rate on plateau
    EarlyStopping(patience=7, restore_best_weights=True),  # Early stopping to prevent overfitting
    ModelCheckpoint(filepath='model-conv1d.h5', save_best_only=True)
]

history = model.fit(X_train, y_train,
                    epochs=70,  # Increased epochs
                    batch_size=32,  # Increased batch size
                    validation_split=0.2,
                    callbacks=callbacks)

Epoch 1/70
4101/4101 [==============================] - 68s 15ms/step - loss: 0.3581 - auc: 0.8889 - val_loss: 0.3545 - val_auc: 0.8952 - lr: 0.0010
Epoch 2/70
4101/4101 [==============================] - 26s 6ms/step - loss: 0.3487 - auc: 0.8979 - val_loss: 0.3555 - val_auc: 0.8928 - lr: 0.0010
Epoch 3/70
4101/4101 [==============================] - 24s 6ms/step - loss: 0.3309 - auc: 0.9098 - val_loss: 0.3625 - val_auc: 0.8884 - lr: 0.0010
Epoch 4/70
4101/4101 [==============================] - 25s 6ms/step - loss: 0.2977 - auc: 0.9275 - val_loss: 0.3948 - val_auc: 0.8827 - lr: 1.0000e-04
Epoch 5/70
4101/4101 [==============================] - 23s 6ms/step - loss: 0.2873 - auc: 0.9323 - val_loss: 0.4070 - val_auc: 0.8789 - lr: 1.0000e-04
Epoch 6/70
4101/4101 [==============================] - 25s 6ms/step - loss: 0.2796 - auc: 0.9360 - val_loss: 0.4142 - val_auc: 0.8777 - lr: 1.0000e-05
Epoch 7/70
4101/4101 [==============================] - 23s 6ms/step - loss: 0.2788 - auc: 0.9364 -

In [29]:
model.save('sub_new.h5')

In [30]:
cnn_model = model
metrics = cnn_model.evaluate(X_test, y_test)
print("{}: {}".format(model.metrics_names[1], metrics[1]))

1282/1282 [==============================] - 3s 2ms/step - loss: 0.3552 - auc: 0.8950
auc: 0.8950294256210327
